<a href="https://colab.research.google.com/github/soyoonjeong/ITCareerfit/blob/main/%EA%B0%80%EC%B9%98%EA%B4%80_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
drive.mount('/content/drive')
pd.options.display.float_format = '{:.0f}'.format

# 500개 기업 데이터 불러오기
data1 = pd.read_csv("/content/drive/MyDrive/CAPSTONE/크롤링/가치관/DART/DART_잡플래닛_20230502.csv", encoding='utf-8', index_col=0)
data2 = pd.read_csv("/content/drive/MyDrive/CAPSTONE/크롤링/가치관/DART/재무제표_수정 (1).csv", encoding='utf-8', thousands=',')
#data1 = pd.read_csv("/content/drive/MyDrive/크롤링/가치관/DART/DART_잡플래닛_20230429_IT_1000_2022.csv", encoding='utf-8', index_col=0)
#data2 = pd.read_csv("/content/drive/MyDrive/크롤링/가치관/DART/재무제표_수정 (1).csv", encoding='utf-8', thousands=',')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.concat([data1, data2], axis=0, ignore_index=True)
data = data[['기업', '기간', '유동자산', '비유동자산', '자산총계', '부채총계', '자본총계', '유동부채', '매출액', '영업이익', '당기순이익']]
# result.to_csv('/content/drive/MyDrive/CAPSTONE/크롤링/DART/DART_합치기_잡플래닛.csv')

In [ ]:
data.head()

,기업,기간,유동자산,비유동자산,자산총계,부채총계,자본총계,유동부채,매출액,영업이익,당기순이익
0,삼성에스디에스,2020.01.01~2020.12.31,4021621788887,2488539129798,6510160918685,803493018170,993400112119,5516760806566,4549427072244,582117407418,266592370964
1,삼성에스디에스,2021.01.01~2021.12.31,4152700905521,2698229150015,6850930055536,970445281046,1236281122384,5614648933152,4985712694305,406734299461,327598640679
2,삼성에스디에스,2022.01.01~2022.12.31,4094239955950,3534659478278,7628899434228,1067177194193,1614217739856,6014681694372,5174554117525,296153499254,530145587281
3,더존비즈온,2020.01.01~2020.12.31,124254793589,671353643425,795608437014,122342749074,403739987480,391868449534,303185154447,75309028866,53753898280
4,더존비즈온,2021.01.01~2021.12.31,207528293324,684579237884,892107531208,388699454466,401654191542,490453339666,313777222994,70088545732,52139958620


In [ ]:
len(data['기업'].unique())

132

In [ ]:
data['연도']=data['기간'].str[0:4]

In [ ]:
# final_df = pd.DataFrame()
# final_df['기업'] = list(data['기업'].unique())
# final_df['수익성'] = 0
# final_df['안정성'] = 0
# final_df['성장성'] = 0

In [ ]:
# final_df.head()

# 수익성

In [ ]:
# 매출액순이익률
data['매출액순이익률'] = (data['당기순이익'] / data['매출액']) *100

# 매출액영업이익률
data['매출액영업이익률'] = (data['영업이익'] / data['매출액']) *100

# #총자산이익률(ROA)
# data['ROA'] = (data['당기순이익'] / data['자산총계']) *100

#자본금이익률(ROE)
# average_equity = data['자본총계'].rolling(2).mean() # 평균 자기 자본
# data['ROE'] = (data['당기순이익'] / average_equity) *100
data['ROE'] = (data['당기순이익'] / data['자본총계']) *100

In [ ]:
from sklearn.preprocessing import MinMaxScaler

profit = data[['매출액순이익률','매출액영업이익률','ROE']]

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(profit)

# 데이터 변환
mMscaled_data = mMscaler.transform(profit)

# 데이터 프레임으로 저장
mMscaled_data = pd.DataFrame(mMscaled_data, columns = ['매출액순이익률','매출액영업이익률','ROE'])
mMscaled_data = mMscaled_data * 100

In [ ]:
profit = pd.concat([data[['기업','연도']], mMscaled_data], axis = 1)
profit['점수'] = (profit['매출액순이익률']+profit['매출액영업이익률']+profit['ROE'])/3
def count_weights(row):
  if row['연도']=='2020':
    return 0.2*3
  elif row['연도']=='2021':
    return 0.3*3
  else:
    return 0.5*3

profit['weight'] = profit.apply(count_weights, axis = 1)

In [ ]:
profit.head()

,기업,연도,매출액순이익률,매출액영업이익률,ROE,점수,weight
0,삼성에스디에스,2020,61,83,76,73,1
1,삼성에스디에스,2021,61,81,76,73,1
2,삼성에스디에스,2022,62,80,76,73,2
3,더존비즈온,2020,63,88,75,75,1
4,더존비즈온,2021,63,87,75,75,1


In [ ]:
profit[profit['기업'].isin(['넥슨코리아','웹젠'])]

,기업,연도,매출액순이익률,매출액영업이익률,ROE,점수,weight
48,웹젠,2020,64,92,79,78,1
49,웹젠,2021,65,93,79,79,1
50,웹젠,2022,66,94,81,80,2
362,넥슨코리아,2020,63,90,75,76,1
363,넥슨코리아,2021,100,88,79,89,1
364,넥슨코리아,2022,63,86,74,74,2


In [ ]:
import numpy as np

grouped = profit.groupby('기업')
weighted_avg_func = lambda g:np.average(g['점수'], weights=g['weight'])
final_profit= pd.DataFrame(grouped.apply(weighted_avg_func))
final_profit.reset_index(inplace = True)
final_profit['수익성 등수'] = len(final_profit) + 1 - final_profit[0].rank()
final_profit['수익성 백분위'] = final_profit['수익성 등수']/ len(final_profit) * 100
final_profit.columns = ['기업', '수익성 점수', '수익성 등수', '수익성 백분위']

In [ ]:
final_profit.head()

,기업,수익성 점수,수익성 등수,수익성 백분위
0,KT아이에스,71.43343,87.00000,65.90909
1,SK텔레콤,72.45996,59.00000,44.69697
2,가비아,72.66423,56.00000,42.42424
3,갤럭시아머니트리,71.98167,70.00000,53.03030
4,골프존,76.09518,18.00000,13.63636


In [ ]:
final_profit.sort_values(by='수익성 백분위')

,기업,수익성 점수,수익성 등수,수익성 백분위
88,인포바인,81.07364,1.00000,0.75758
127,헥토이노베이션,79.74536,2.00000,1.51515
67,엠게임,79.64136,3.00000,2.27273
45,시큐브,79.59287,4.00000,3.03030
76,웹젠,79.50222,5.00000,3.78788
...,...,...,...,...
36,브랜디,60.18844,128.00000,96.96970
60,에이트원,52.89711,129.00000,97.72727
30,룽투코리아,48.25642,130.00000,98.48485
35,베스파,46.82051,131.00000,99.24242


In [ ]:
final_profit['profit_cut']=pd.cut(final_profit['수익성 백분위'], bins=[0, 4, 11, 23, 40, 60, 100], labels=['A', 'B','C', 'D', 'E', 'F'] )

In [ ]:
final_profit['4%'] = np.abs(final_profit['수익성 백분위']-4)
final_profit['11%'] = np.abs(final_profit['수익성 백분위']-11)
final_profit['23%'] = np.abs(final_profit['수익성 백분위']-23)
final_profit['40%'] = np.abs(final_profit['수익성 백분위']-40)
final_profit['60%'] = np.abs(final_profit['수익성 백분위']-60)

In [ ]:
pd.options.display.max_rows=None
final_profit.sort_values(by='수익성 등수')

,기업,수익성 점수,수익성 등수,수익성 백분위,profit_cut,4%,11%,23%,40%,60%
88,인포바인,81,1,1,A,3,10,22,39,59
127,헥토이노베이션,80,2,2,A,2,9,21,38,58
67,엠게임,80,3,2,A,2,9,21,38,58
45,시큐브,80,4,3,A,1,8,20,37,57
76,웹젠,80,5,4,A,0,7,19,36,56
13,넥슨코리아,79,6,5,B,1,6,18,35,55
96,커넥트웨이브,79,7,5,B,1,6,18,35,55
116,포시에스,78,8,6,B,2,5,17,34,54
125,한글과컴퓨터,78,9,7,B,3,4,16,33,53
39,사람인,78,10,8,B,4,3,15,32,52


In [ ]:
final_profit[final_profit['profit_cut']=='A']

,기업,수익성 점수,수익성 등수,수익성 백분위,profit_cut
45,시큐브,79.59287,4.00000,3.03030,A
67,엠게임,79.64136,3.00000,2.27273,A
76,웹젠,79.50222,5.00000,3.78788,A
88,인포바인,81.07364,1.00000,0.75758,A
127,헥토이노베이션,79.74536,2.00000,1.51515,A


In [ ]:
final_profit[final_profit['profit_cut']=='B']

,기업,수익성 점수,수익성 등수,수익성 백분위,profit_cut
10,네이버,76.92827,12.00000,9.09091,B
13,넥슨코리아,79.09798,6.00000,4.54545,B
39,사람인,77.73193,10.00000,7.57576,B
51,아이퀘스트,76.79203,13.00000,9.84848,B
55,아프리카티비,76.63054,14.00000,10.60606,B
96,커넥트웨이브,79.06312,7.00000,5.30303,B
98,케이아이엔엑스,77.02367,11.00000,8.33333,B
116,포시에스,78.35148,8.00000,6.06061,B
125,한글과컴퓨터,77.91556,9.00000,6.81818,B


# 안정성

In [ ]:
# 유동성비율
data['유동성비율'] = data['유동자산'] / data['유동부채'] * 100

# 당좌비율
# data['당좌비율'] = (data['유동자산']-data['재고자산'])/data['유동부채'] * 100

# 부채비율
data['부채비율'] = -data['부채총계'] / data['자본총계'] * 100

# 자기자본비율
data['자기자본비율'] = data['자본총계'] / data['자산총계'] * 100 # average_equity: 평균 자기 자본

In [ ]:
data[data['기업']=='네이버'][['기업', '연도', '유동성비율','부채비율','자기자본비율']]

,기업,연도,유동성비율,부채비율,자기자본비율
315,네이버,2020,13,-81,18
316,네이버,2021,19,-37,28
317,네이버,2022,21,-37,29


In [ ]:
from sklearn.preprocessing import MinMaxScaler

stable = data[['유동성비율','부채비율','자기자본비율']]

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(stable)

# 데이터 변환
mMscaled_data = mMscaler.transform(stable)

# 데이터 프레임으로 저장
mMscaled_data = pd.DataFrame(mMscaled_data, columns = ['유동성비율','부채비율','자기자본비율'])
mMscaled_data = mMscaled_data * 100

In [ ]:
stable = pd.concat([data[['기업','연도']], mMscaled_data], axis = 1)
stable['점수'] = (stable['유동성비율']+stable['부채비율']+stable['자기자본비율'])/3
def count_weights(row):
  if row['연도']=='2020':
    return 0.1*3
  elif row['연도']=='2021':
    return 0.3*3
  else:
    return 0.6*3

stable['weight'] = stable.apply(count_weights, axis = 1)

In [ ]:
stable.head()

,기업,연도,유동성비율,부채비율,자기자본비율,점수,weight
0,삼성에스디에스,2020,41,97,1,46,0
1,삼성에스디에스,2021,41,97,2,46,1
2,삼성에스디에스,2022,40,97,2,47,2
3,더존비즈온,2020,37,99,6,47,0
4,더존비즈온,2021,38,96,5,46,1


In [ ]:
import numpy as np

grouped = stable.groupby('기업')
weighted_avg_func = lambda g:np.average(g['점수'], weights=g['weight'])
final_stable= pd.DataFrame(grouped.apply(weighted_avg_func))
final_stable.reset_index(inplace = True)
final_stable['안정성 등수'] = len(final_stable) + 1 - final_stable[0].rank()
final_stable['안정성 백분위'] = final_stable['안정성 등수']/ len(final_stable) * 100
final_stable.columns = ['기업', '안정성 점수', '안정성 등수', '안정성 백분위']

In [ ]:
final_stable.sort_values(by='안정성 백분위')

,기업,안정성 점수,안정성 등수,안정성 백분위
35,베스파,64,1,1
13,넥슨코리아,63,2,2
74,와이더플래닛,62,3,2
103,코나아이,58,4,3
121,하이브랩,57,5,4
39,사람인,54,6,5
72,오파스넷,54,7,5
34,바이브컴퍼니,53,8,6
3,갤럭시아머니트리,53,9,7
27,라이프시맨틱스,52,10,8


In [ ]:
final_stable['4%'] = np.abs(final_stable['안정성 백분위']-4)
final_stable['11%'] = np.abs(final_stable['안정성 백분위']-11)
final_stable['23%'] = np.abs(final_stable['안정성 백분위']-23)
final_stable['40%'] = np.abs(final_stable['안정성 백분위']-40)
final_stable['60%'] = np.abs(final_stable['안정성 백분위']-60)

In [ ]:
final_stable.sort_values(by='안정성 등수')

,기업,안정성 점수,안정성 등수,안정성 백분위,4%,11%,23%,40%,60%
35,베스파,64,1,1,3,10,22,39,59
13,넥슨코리아,63,2,2,2,9,21,38,58
74,와이더플래닛,62,3,2,2,9,21,38,58
103,코나아이,58,4,3,1,8,20,37,57
121,하이브랩,57,5,4,0,7,19,36,56
39,사람인,54,6,5,1,6,18,35,55
72,오파스넷,54,7,5,1,6,18,35,55
34,바이브컴퍼니,53,8,6,2,5,17,34,54
3,갤럭시아머니트리,53,9,7,3,4,16,33,53
27,라이프시맨틱스,52,10,8,4,3,15,32,52


# 성장성

In [ ]:
data2=pd.read_csv("/content/drive/MyDrive/CAPSTONE/크롤링/DART/company백업_20230510.csv", encoding='utf-8', index_col=0)
data2

FileNotFoundError: ignored

In [ ]:
import numpy as np
# 영업이익증가율
data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['영업이익증가율'] = (data['영업이익'].diff() / data['영업이익'] )*100
data.loc[data['연도'] == 2020, '영업이익증가율'] = np.nan

# 매출액증가율
data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['매출액증가율'] = (data['매출액'].diff() / data['매출액'])*100
data.loc[data['연도'] == 2020, '매출액증가율'] = np.nan

#순이익증가율
data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['당기순이익증가율'] = (data['당기순이익'].diff() / data['당기순이익'])*100
data.loc[data['연도'] == 2020, '당기순이익증가율'] = np.nan

In [ ]:
data[data['기업']=='KT아이에스']

,기업,기간,유동자산,비유동자산,자산총계,부채총계,자본총계,유동부채,매출액,영업이익,당기순이익,연도,영업이익증가율,매출액증가율,당기순이익증가율
324,KT아이에스,2020.01.01~2020.12.31,126275205062.00000,168013679695.00000,294288884757.00000,87387458189.00000,126893637851.00000,167395246906.00000,453639144631.00000,9002862020.00000,7387478525.00000,2020,NaN,NaN,NaN
325,KT아이에스,2021.01.01~2021.12.31,124420074786.00000,244940730764.00000,369360805550.00000,103927321508.00000,177618511970.00000,191742293580.00000,468004263986.00000,14405202066.00000,24944162467.00000,2021,37.50270,3.06944,70.38394
326,KT아이에스,2022.01.01~2022.12.31,102120905227.00000,294087164063.00000,396208069290.00000,103697864970.00000,199203706481.00000,197004362809.00000,535783305933.00000,19544619038.00000,15916623698.00000,2022,26.29582,12.65046,-56.71767


In [ ]:
from sklearn.preprocessing import MinMaxScaler

new_data = data[data['연도'].isin(['2021','2022'])][['기업', '연도', '영업이익증가율','매출액증가율','당기순이익증가율']]
# grow = new_data[['영업이익증가율','매출액증가율','당기순이익증가율']]

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(new_data[['영업이익증가율','매출액증가율','당기순이익증가율']] )

# 데이터 변환
mMscaled_data = mMscaler.transform(new_data[['영업이익증가율','매출액증가율','당기순이익증가율']] )

# 데이터 프레임으로 저장
mMscaled_data = pd.DataFrame(mMscaled_data, columns = ['영업이익증가율','매출액증가율','당기순이익증가율'])
mMscaled_data = mMscaled_data * 100

In [ ]:
grow = pd.concat([new_data[['기업','연도']], mMscaled_data], axis = 1)
grow['점수'] = (grow['영업이익증가율']+grow['매출액증가율']+grow['당기순이익증가율'])/3
def count_weights(row):
  if row['연도']=='2021':
    return 0.3*2
  else:
    return 0.7*2
grow['weight'] = grow.apply(count_weights, axis = 1)

In [ ]:
len(grow['기업'].unique())

133

In [ ]:
grow.head(10)

,기업,연도,영업이익증가율,매출액증가율,당기순이익증가율
325,KT아이에스,2021,NaN,NaN,NaN
326,KT아이에스,2022,NaN,NaN,NaN
13,SK텔레콤,2021,58.23153,88.33023,24.60830
14,SK텔레콤,2022,58.40571,88.61953,24.82390
79,가비아,2021,58.23145,88.99854,24.86986
80,가비아,2022,57.65503,88.49872,24.86371
175,갤럭시아머니트리,2021,60.39991,70.11887,35.86526
176,갤럭시아머니트리,2022,57.79895,87.50891,24.55475
55,골프존,2021,58.10488,79.27831,23.86424
56,골프존,2022,57.78500,89.29005,29.80225


In [ ]:
import numpy as np

grouped = grow.groupby('기업')
weighted_avg_func = lambda g:np.average(g['점수'], weights=g['weight'])
final_grow= pd.DataFrame(grouped.apply(weighted_avg_func))
final_grow.reset_index(inplace = True)
final_grow['성장성 등수'] = len(final_grow) + 1 - final_grow[0].rank()
final_grow['성장성 백분위'] = final_grow['성장성 등수']/ len(final_grow) * 100
final_grow.columns = ['기업', '성장성 점수', '성장성 등수', '성장성 백분위']
final_grow['성장성 점수'] = final_grow['성장성 점수']

In [ ]:
final_grow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   기업       132 non-null    object 
 1   성장성 점수   87 non-null     float64
 2   성장성 등수   87 non-null     float64
 3   성장성 백분위  87 non-null     float64
dtypes: float64(3), object(1)
memory usage: 4.2+ KB


In [ ]:
grow[grow['기업']=='쿠팡']

,기업,연도,영업이익증가율,매출액증가율,당기순이익증가율,점수,weight
378,쿠팡,2021,NaN,NaN,NaN,NaN,0.60000
379,쿠팡,2022,NaN,NaN,NaN,NaN,1.40000


In [ ]:
grow.describe()

,영업이익증가율,매출액증가율,당기순이익증가율,점수,weight
count,263.00000,263.00000,263.00000,263.00000,263.00000
mean,58.37204,87.94978,25.33782,57.21988,0.99848
std,5.23115,6.65878,7.08403,4.09493,0.40076
min,0.00000,0.00000,0.00000,23.55656,0.60000
25%,58.15769,87.43989,24.61210,56.78765,0.60000
50%,58.41674,88.49391,24.98238,57.29794,0.60000
75%,58.82504,89.81884,25.44587,58.03111,1.40000
max,100.00000,100.00000,100.00000,85.62192,1.40000


In [ ]:
final_grow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   기업       132 non-null    object 
 1   성장성 점수   132 non-null    float64
 2   성장성 등수   132 non-null    float64
 3   성장성 백분위  132 non-null    float64
dtypes: float64(3), object(1)
memory usage: 4.2+ KB


In [ ]:
final_grow[final_grow['기업']=='삼성에스디에스']

,기업,성장성 점수,성장성 등수,성장성 백분위
40,삼성에스디에스,57.07568,100.00000,75.75758


In [ ]:
final_grow.head()

,기업,성장성 점수,성장성 등수,성장성 백분위
0,KT아이에스,NaN,NaN,NaN
1,SK텔레콤,57.21514,94.00000,71.21212
2,가비아,57.11406,98.00000,74.24242
3,갤럭시아머니트리,56.27301,123.00000,93.18182
4,골프존,57.39611,86.00000,65.15152


In [ ]:
final_grow.describe()

,성장성 점수,성장성 등수,성장성 백분위
count,132.00000,132.00000,132.00000
mean,57.13014,66.50000,50.37879
std,3.39531,38.24918,28.97665
min,32.87841,1.00000,0.75758
25%,56.74356,33.75000,25.56818
50%,57.25122,66.50000,50.37879
75%,57.76722,99.25000,75.18939
max,71.04666,132.00000,100.00000


In [ ]:
final_grow.sort_values(by='성장성 백분위')

,기업,성장성 점수,성장성 등수,성장성 백분위
64,엔텔스,71.04666,1.00000,0.75758
49,쌍용정보통신,66.37854,2.00000,1.51515
105,쿠팡,65.06683,3.00000,2.27273
90,인피니트헬스케어,63.97133,4.00000,3.03030
32,무신사,62.63259,5.00000,3.78788
...,...,...,...,...
83,유엔젤,52.38002,128.00000,96.96970
84,이니텍,51.71712,129.00000,97.72727
119,플레이위드,50.75215,130.00000,98.48485
89,인포뱅크,40.97629,131.00000,99.24242


In [ ]:
from sklearn.preprocessing import MinMaxScaler

new_data = data[data['연도'].isin(['2021','2022'])][['기업', '연도', '영업이익증가율','매출액증가율','당기순이익증가율']]
grow = new_data[['영업이익증가율','매출액증가율','당기순이익증가율']]

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(grow[['영업이익증가율','매출액증가율','당기순이익증가율']])

# 데이터 변환
mMscaled_data = mMscaler.transform(grow[['영업이익증가율','매출액증가율','당기순이익증가율']])

# 데이터 프레임으로 저장
grow[['영업이익증가율','매출액증가율','당기순이익증가율']] = mMscaled_data * 100
# mMscaled_data = pd.DataFrame(mMscaled_data, columns = ['영업이익증가율','매출액증가율','당기순이익증가율'])
# mMscaled_data = mMscaled_data * 100

# 규모 형태

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/크롤링/기업목록/잡플래닛 기업데이터_20230506.csv")
df = df[['cp_name', 'cp_type', 'emp_num']]
df.head()

FileNotFoundError: ignored

In [ ]:
def count_cp_type_score(row):
  if row['cp_type'] == '대기업':
    return 100
  elif row['cp_type'] == '중견기업':
    return 75
  elif row['cp_type'] == '중소기업':
    return 25
  else:
    return 50

df['cp_type_score'] = df.apply(count_cp_type_score, axis= 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df['emp_num'].fillna(np.mean(df['emp_num']), inplace = True)
# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(df[['emp_num']])

# 데이터 변환
mMscaled_data = mMscaler.transform(df[['emp_num']])

# 데이터 프레임으로 저장
df[['emp_num']] = mMscaled_data * 100

In [ ]:
df['scale_score'] = (df['cp_type_score'] + df['emp_num'])/2
df['scale_rank'] = len(df) + 1 - df['scale_score'].rank()
df['scale_per'] = round(100 - df['scale_rank']/ len(df) * 100,2)

In [ ]:
df.head()

,cp_name,cp_type,emp_num,cp_type_score,scale_score,scale_rank,scale_per
0,삼성에스디에스,대기업,100.00000,100,100.00000,1.00000,99.24000
1,더존비즈온,대기업,12.04529,100,56.02265,11.00000,91.67000
2,롯데정보통신,대기업,6.82118,100,53.41059,13.00000,90.15000
3,카페24,중견기업,9.81290,75,42.40645,36.00000,72.73000
4,SK텔레콤,대기업,42.48775,100,71.24388,7.00000,94.70000


In [ ]:
data = []
for idx in range(len(df)):
  row = df.iloc[idx]
  data.append((row['scale_per'],row['cp_name']))

# 연봉

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/CAPSTONE/크롤링/기업목록/잡플래닛 기업데이터_20230506.csv")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df = df[['cp_name','pay']]

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(df[['pay']])

# 데이터 변환
mMscaled_data = mMscaler.transform(df[['pay']])

# 데이터 프레임으로 저장
df[['pay_score']] = mMscaled_data * 100
df['pay_rank'] = len(df) + 1 - df['pay_score'].rank()
df['pay_per'] = round(df['pay_rank']/ len(df) * 100,2)

<ipython-input-39-321ed1d2ccab>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['pay_score']] = mMscaled_data * 100
<ipython-input-39-321ed1d2ccab>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pay_rank'] = len(df) + 1 - df['pay_score'].rank()


In [ ]:
df.sort_values(by='pay_rank')

,cp_name,pay,pay_score,pay_rank,pay_per,4%,11%,23%,40%,60%
127,페이스북코리아,8688,100,1,1,3,10,22,39,59
128,구글코리아,7008,71,2,2,2,9,21,38,58
110,나이스평가정보,6927,69,3,2,2,9,21,38,58
4,SK텔레콤,6652,65,4,3,1,8,20,37,57
94,나이스정보통신,5964,53,5,4,0,7,19,36,56
105,네이버,5908,52,6,5,1,6,18,35,55
0,삼성에스디에스,5906,52,7,5,1,6,18,35,55
118,카카오,5467,44,8,6,2,5,17,34,54
8,현대오토에버,5457,44,9,7,3,4,16,33,53
109,엘에스산전,5236,40,10,8,4,3,15,32,52


In [ ]:
df['4%'] = np.abs(df['pay_per']-4)
df['11%'] = np.abs(df['pay_per']-11)
df['23%'] = np.abs(df['pay_per']-23)
df['40%'] = np.abs(df['pay_per']-40)
df['60%'] = np.abs(df['pay_per']-60)

In [ ]:
df

,cp_name,pay,pay_score,pay_rank,pay_per,4%,11%,23%,40%,60%
0,삼성에스디에스,5906,52,7,5,1,6,18,35,55
1,더존비즈온,3986,18,76,58,54,47,35,18,2
2,롯데정보통신,4771,32,22,17,13,6,6,23,43
3,카페24,4312,24,46,35,31,24,12,5,25
4,SK텔레콤,6652,65,4,3,1,8,20,37,57
5,안랩,4380,25,40,30,26,19,7,10,30
6,케이티알파,4691,31,24,18,14,7,5,22,42
7,엔에이치엔,4889,34,18,14,10,3,9,26,46
8,현대오토에버,5457,44,9,7,3,4,16,33,53
9,다우기술,4652,30,29,22,18,11,1,18,38


In [ ]:
df

,기업,수익성 점수,수익성 등수,수익성 백분위,profit_cut,4%,11%,23%,40%,60%
0,KT아이에스,71,87,66,F,62,55,43,26,6
1,SK텔레콤,72,59,45,E,41,34,22,5,15
2,가비아,73,56,42,E,38,31,19,2,18
3,갤럭시아머니트리,72,70,53,E,49,42,30,13,7
4,골프존,76,18,14,C,10,3,9,26,46
5,구글코리아,72,63,48,E,44,37,25,8,12
6,나이스정보통신,74,40,30,D,26,19,7,10,30
7,나이스평가정보,74,32,24,D,20,13,1,16,36
8,네오리진,69,113,86,F,82,75,63,46,26
9,네오위즈,74,35,27,D,23,16,4,13,33


In [ ]:
df.head()

,cp_name,pay,pay_score,pay_rank,pay_per
0,삼성에스디에스,5906,52,7,95
1,더존비즈온,3986,18,76,42
2,롯데정보통신,4771,32,22,83
3,카페24,4312,24,46,65
4,SK텔레콤,6652,65,4,97


In [ ]:
data = []
for idx in range(len(df)):
  row = df.iloc[idx]
  data.append((row['pay_per'],row['cp_name']))

# DB 넣기

In [ ]:
final_df = pd.merge(final_profit, final_stable, how='inner', on=None)
final_df = pd.merge(final_df, final_grow, how = 'inner', on=None)
final_df = final_df[['기업','수익성 백분위','안정성 백분위','성장성 백분위']]
final_df.rename(columns = {'기업':'cp_name',
                 '수익성 백분위':'profit',
                 '안정성 백분위':'stable',
                 '성장성 백분위':'grow'}, inplace = True)
final_df.head()

,cp_name,profit,stable,grow
0,KT아이에스,65.90909,41.66667,50.75758
1,SK텔레콤,44.69697,39.39394,66.66667
2,가비아,42.42424,44.69697,53.03030
3,갤럭시아머니트리,53.03030,6.81818,6.81818
4,골프존,13.63636,58.33333,15.90909


In [ ]:
for indi in ['profit','stable','grow']:
  final_df[indi] = 100-final_df[indi] # 수익성, 안정성, 성장성만
  final_df[indi] = round(final_df[indi],2)

In [ ]:
data = []
for idx in range(len(final_df)):
  row = final_df.iloc[idx]
  data.append((row['profit'],row['stable'],row['grow'],row['cp_name']))

In [ ]:
final_df.head()

In [ ]:
!pip install pymysql

In [ ]:
import numpy as np
import pymysql
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
host = 'itcareerfit.cc5um8ekupdb.ap-northeast-2.rds.amazonaws.com'
port = 3306
user_name = 'admin'
password = 'teamone!'
db = 'itcareerfit'

conn = pymysql.connect(host = 'itcareerfit.cc5um8ekupdb.ap-northeast-2.rds.amazonaws.com',
                        port = 3306,
                        user = 'admin',
                        password = 'teamone!',
                        db = 'itcareerfit')
cursor = conn.cursor()

#query = "UPDATE company SET profit = %s, stable = %s, grow = %s WHERE cp_name = %s"
query = "UPDATE company SET pay = %s WHERE cp_name = %s"
cursor.executemany(query, data)
conn.commit()
cursor.close()
conn.close()